In [15]:
import tech_indicators as ti
import pandas as pd
import numpy as np
import datetime as dt
from statsmodels.tsa.seasonal import STL

stock_list = ['AAPL','MSFT','GOOG','AMZN','NVDA','BRK-B','TSLA','META','JNJ','V']

df_spy_daily = pd.read_csv("../stock_price/Daily/SPY.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'SPY'})
df_spy_daily['Date'] = pd.to_datetime(df_spy_daily['Date'])
df_spy_monthly = pd.read_csv("../stock_price/Monthly/SPY.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'SPY'})
df_spy_monthly['Date'] = pd.to_datetime(df_spy_monthly['Date'])
df_spy_weekly = pd.read_csv("../stock_price/Weekly/SPY.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'SPY'})
df_spy_weekly['Date'] = pd.to_datetime(df_spy_weekly['Date'])

df_ndaq_daily = pd.read_csv("../stock_price/Daily/NDAQ.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'NDAQ'})
df_ndaq_daily['Date'] = pd.to_datetime(df_ndaq_daily['Date'])
df_ndaq_monthly = pd.read_csv("../stock_price/Monthly/NDAQ.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'NDAQ'})
df_ndaq_monthly['Date'] = pd.to_datetime(df_ndaq_monthly['Date'])
df_ndaq_weekly = pd.read_csv("../stock_price/Weekly/NDAQ.csv").loc[:,['Date','adjusted_close']].rename(columns={'adjusted_close':'NDAQ'})
df_ndaq_weekly['Date'] = pd.to_datetime(df_ndaq_weekly['Date'])

df_ndaq_daily 

,Date,NDAQ
0,2007-01-03,8.575854
1,2007-01-04,8.819220
2,2007-01-05,9.153847
3,2007-01-08,9.413805
4,2007-01-09,9.355729
...,...,...
3288,2020-01-27,34.929343
3289,2020-01-28,35.362062
3290,2020-01-29,36.698400
3291,2020-01-30,36.895669


In [7]:
def calculate_mfi(df,period=14):
    """
    Calculates the money flow index (MFI) of a DataFrame.
    Parameters:
    period (int): Number of periods to use for the MFI calculation.
    Returns:
    pandas.DataFrame: DataFrame containing the MFI values.
    """
    #df = get_daily_adjusted_prices(symbol)
    df["close"] = pd.to_numeric(df["close"])
    df["low"] = pd.to_numeric(df["low"])
    df["high"] = pd.to_numeric(df["high"])
    df["volume"] = pd.to_numeric(df["volume"])
    # Calculate typical price
    typical_price = (df['close'] + df['low'] + df['high']) / 3

    # Calculate raw money flow
    money_flow = typical_price * df['volume']

    # Get positive and negative money flow
    positive_flow = np.where(typical_price > typical_price.shift(1).fillna(method='bfill'), money_flow, 0)
    negative_flow = np.where(typical_price < typical_price.shift(1).fillna(method='bfill'), money_flow, 0)

    # Calculate the money flow ratio
    positive_flow,negative_flow = pd.Series(positive_flow).rolling(window=period).sum(),pd.Series(negative_flow).rolling(window=period).sum()
    raw_mfr = positive_flow / negative_flow
    mfi = pd.Series(np.where(negative_flow == 0, 100, 100 - (100 / (1 + raw_mfr))))

    # Calculate the MFI
    #mfi = mfr.rolling(window=period).mean()

    # Add MFI values to the DataFrame
    df['mfi'] = mfi

    return df.loc[:,'mfi']

In [17]:
for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Daily/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df.reset_index(inplace=True)
    sma = ti.calculate_sma(df, window=14)
    ema = ti.calculate_ema(df, window=14)
    k, d = ti.calculate_stoch(df)
    rsi = ti.calculate_rsi(df)
    mfi = calculate_mfi(df)
    sar = ti.calculate_sar(df)
    ad = ti.calc_AD(df)

    tech_Daily = pd.DataFrame({"Date":df.loc[:,"Date"],
              "SMA":sma,
              "EMA":ema,
              "STOCH_k":k,
              "STOCK_d":d,
              "RSI":rsi,
              "MFI":mfi,
              "SAR":sar,
              "AD":ad
              })
    tech_Daily['Date'] = pd.to_datetime(tech_Daily['Date'])
    tech_Daily = tech_Daily.loc[(tech_Daily['Date']>='2010-01-01') & (tech_Daily['Date']<='2020-02-01')]
    tech_Daily = tech_Daily.reset_index(drop=True)
    tech_Daily = pd.merge(tech_Daily,df_spy_daily,on='Date',how='left')
    tech_Daily = pd.merge(tech_Daily,df_ndaq_daily,on='Date',how='left')
    tech_Daily.to_csv("Tech/Daily/"+stock+".csv",index=0)
tech_Daily 

,Date,SMA,EMA,STOCH_k,STOCK_d,RSI,MFI,SAR,AD,SPY,NDAQ
0,2010-01-04,87.050714,86.469171,8800.533871,8780.517509,74.637681,79.040088,2.029269e+04,1.290264e+09,88.050132,5.597397
1,2010-01-05,87.219286,86.557282,8698.509499,8745.184233,60.369069,73.772836,2.069679e+04,1.218850e+09,88.283213,5.613990
2,2010-01-06,87.206429,86.477644,8575.594272,8691.545881,49.194991,70.378941,2.110901e+04,1.151087e+09,88.345367,5.611225
3,2010-01-07,87.187143,86.515292,8655.594272,8643.232681,48.782687,70.096512,2.152947e+04,1.223507e+09,88.718297,5.536556
4,2010-01-08,87.185000,86.579919,8679.335749,8636.841431,49.867608,75.080973,2.195835e+04,1.296902e+09,89.013532,5.594632
...,...,...,...,...,...,...,...,...,...,...,...
2532,2020-01-27,199.490000,200.234024,20160.240130,20431.981364,74.112654,80.031356,1.205512e+26,5.225094e+10,307.043805,34.929343
2533,2020-01-28,200.501429,200.582821,20276.148617,20309.413576,76.636569,80.540753,1.229622e+26,5.285468e+10,310.261359,35.362062
2534,2020-01-29,201.425714,201.153111,20475.067575,20303.818774,75.512618,80.810211,1.254215e+26,5.346109e+10,310.005094,36.698400
2535,2020-01-30,202.494286,202.094030,20809.653937,20520.290043,77.319211,81.152525,1.279299e+26,5.407506e+10,311.011172,36.895669


In [18]:
for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Weekly/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df.reset_index(inplace=True)
    sma = ti.calculate_sma(df, window=14)
    ema = ti.calculate_ema(df, window=14)
    k, d = ti.calculate_stoch(df)
    rsi = ti.calculate_rsi(df)
    mfi = calculate_mfi(df)
    sar = ti.calculate_sar(df)
    ad = ti.calc_AD(df)

    tech_Weekly = pd.DataFrame({"Date":df.loc[:,"Date"],
              "SMA":sma,
              "EMA":ema,
              "STOCH_k":k,
              "STOCK_d":d,
              "RSI":rsi,
              "MFI":mfi,
              "SAR":sar,
              "AD":ad
              })
    tech_Weekly['Date'] = pd.to_datetime(tech_Weekly['Date'])
    tech_Weekly = tech_Weekly.loc[(tech_Weekly['Date']>='2010-01-01') & (tech_Weekly['Date']<='2020-02-01')]
    tech_Weekly = tech_Weekly.reset_index(drop=True)
    tech_Weekly = pd.merge(tech_Weekly,df_spy_weekly,on='Date',how='left')
    tech_Weekly = pd.merge(tech_Weekly,df_ndaq_weekly,on='Date',how='left')
    tech_Weekly.to_csv("Tech/Weekly/"+stock+".csv",index=0)
tech_Weekly 

,Date,SMA,EMA,STOCH_k,STOCK_d,RSI,MFI,SAR,AD,SPY,NDAQ
0,2010-01-08,80.602857,81.179788,8696.976223,8688.741962,87.013487,82.878940,60.720900,3.861826e+09,89.0135,5.5946
1,2010-01-15,81.545714,81.841149,8609.794544,8683.298533,80.470914,77.778757,61.281882,3.371076e+09,88.2910,5.5753
2,2010-01-22,82.148571,81.959663,8268.794544,8525.188437,67.790894,72.713330,61.815045,2.894890e+09,84.8492,5.0720
3,2010-01-29,82.713571,81.969041,8198.794544,8359.127878,66.309278,68.044877,62.346944,2.425818e+09,83.4351,4.9752
4,2010-02-05,83.199286,82.047836,8250.542837,8239.377308,64.693172,67.422754,62.799605,2.882981e+09,82.8680,5.1992
...,...,...,...,...,...,...,...,...,...,...,...
521,2020-01-03,181.753571,183.104288,18952.523725,18891.666128,78.363636,67.088343,155.217186,1.200533e+11,306.0093,34.4521
522,2020-01-10,183.024286,184.526383,19370.022550,19084.543751,79.959582,76.184441,155.935642,1.227405e+11,309.1414,33.9939
523,2020-01-17,184.998571,187.216199,20465.077235,19595.874503,84.951947,77.424269,156.708329,1.254778e+11,315.0640,34.7798
524,2020-01-24,187.090714,189.587372,20495.740175,20110.279987,88.048844,78.123002,157.669162,1.281907e+11,312.0457,35.0662


In [20]:
for stock in stock_list:
    stock_df = pd.read_csv("../stock_price/Monthly/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df.reset_index(inplace=True)
    sma = ti.calculate_sma(df, window=14)
    ema = ti.calculate_ema(df, window=14)
    k, d = ti.calculate_stoch(df)
    rsi = ti.calculate_rsi(df)
    mfi = calculate_mfi(df)
    sar = ti.calculate_sar(df)
    ad = ti.calc_AD(df)

    tech_Monthly = pd.DataFrame({"Date":df.loc[:,"Date"],
              "SMA":sma,
              "EMA":ema,
              "STOCH_k":k,
              "STOCK_d":d,
              "RSI":rsi,
              "MFI":mfi,
              "SAR":sar,
              "AD":ad
              })
    tech_Monthly['Date'] = pd.to_datetime(tech_Monthly['Date'])
    tech_Monthly = tech_Monthly.loc[(tech_Monthly['Date']>='2010-01-01') & (tech_Monthly['Date']<='2020-02-01')]
    tech_Monthly = tech_Monthly.reset_index(drop=True)
    tech_Monthly = pd.merge(tech_Monthly,df_spy_monthly,on='Date',how='left')
    tech_Monthly = pd.merge(tech_Monthly,df_ndaq_monthly,on='Date',how='left')
    tech_Monthly.to_csv("Tech/Monthly/"+stock+".csv",index=0)
tech_Monthly 

,Date,SMA,EMA,STOCH_k,STOCK_d,RSI,MFI,SAR,AD,SPY,NDAQ
0,2010-01-29,67.211429,72.957338,8202.127948,8348.738778,73.077525,65.332522,46.421880,2.338943e+09,83.4351,4.9752
1,2010-02-26,69.556429,74.600360,8527.127948,8491.461282,74.503657,65.571182,47.268843,3.551670e+09,86.0379,5.1521
2,2010-03-31,72.533571,76.790978,9101.968540,8610.408145,79.925330,71.567999,48.078666,4.808868e+09,91.2780,5.8408
3,2010-04-30,74.927857,78.582848,9021.994056,8883.696848,76.569436,71.092458,48.989693,3.629440e+09,92.6901,5.8076
4,2010-05-28,76.132143,77.766468,7244.768881,8456.243826,60.571859,65.259243,49.953739,2.660060e+09,85.3254,5.1411
...,...,...,...,...,...,...,...,...,...,...,...
116,2019-09-30,155.567143,157.169823,17199.142093,17692.416342,67.103094,60.260111,99.085267,8.965963e+10,280.3014,31.4687
117,2019-10-31,157.850714,160.061847,17884.142093,17721.112551,66.015429,53.563570,100.844562,9.357406e+10,286.4974,31.6018
118,2019-11-29,160.309286,163.321601,18449.142093,17844.142093,66.829650,53.871889,102.431271,9.761312e+10,296.8681,33.1950
119,2019-12-31,163.884286,166.598720,18788.219359,18373.834515,76.790494,62.594108,104.079645,1.017502e+11,305.4872,34.0766


In [26]:
# PCA 
import numpy as np
import pandas as pd
import sys
from scipy.sparse import dia_matrix
import scipy.sparse.linalg
import math
from pca import pca
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from fancyimpute import KNN
from sklearn.impute import KNNImputer

features = list(tech_Daily.columns[1:])
features

['SMA', 'EMA', 'STOCH_k', 'STOCK_d', 'RSI', 'MFI', 'SAR', 'AD', 'SPY', 'NDAQ']

In [28]:
# small test 
def knnpca(df, k=5, n_components = 3):
    x = df.loc[:,features].values
    x = StandardScaler().fit_transform(x)
    x = KNN(k).fit_transform(x)
    pca = PCA(n_components)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents
                , columns = ['PC1', 'PC2','PC3'])
    
    return principalDf

knnpca(tech_Daily)

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


Imputing row 1/2537 with 0 missing, elapsed time: 0.529
Imputing row 101/2537 with 0 missing, elapsed time: 0.529
Imputing row 201/2537 with 0 missing, elapsed time: 0.529
Imputing row 301/2537 with 0 missing, elapsed time: 0.530
Imputing row 401/2537 with 0 missing, elapsed time: 0.530
Imputing row 501/2537 with 0 missing, elapsed time: 0.530
Imputing row 601/2537 with 0 missing, elapsed time: 0.530
Imputing row 701/2537 with 0 missing, elapsed time: 0.530
Imputing row 801/2537 with 0 missing, elapsed time: 0.530
Imputing row 901/2537 with 0 missing, elapsed time: 0.530
Imputing row 1001/2537 with 0 missing, elapsed time: 0.530
Imputing row 1101/2537 with 0 missing, elapsed time: 0.530
Imputing row 1201/2537 with 0 missing, elapsed time: 0.531
Imputing row 1301/2537 with 0 missing, elapsed time: 0.531
Imputing row 1401/2537 with 0 missing, elapsed time: 0.531
Imputing row 1501/2537 with 0 missing, elapsed time: 0.531
Imputing row 1601/2537 with 0 missing, elapsed time: 0.531
Imputing 

,PC1,PC2,PC3
0,-1.938664,-0.261036,-2.420977
1,-2.078862,-0.441699,-1.519838
2,-2.193366,-0.573436,-0.849733
3,-2.193984,-0.580761,-0.817267
4,-2.150416,-0.516532,-1.103537
...,...,...,...
2532,6.782523,4.713786,-0.324626
2533,6.896492,4.850739,-0.430616
2534,7.010524,4.967295,-0.352776
2535,7.151917,5.038326,-0.427326


In [29]:
for stock in stock_list:
    stock_df = pd.read_csv("Tech/Daily/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df = df.loc[:,features]
    df = df.dropna()
    df = df.reset_index(drop=True)
    df_pca = knnpca(df)
    df = pd.concat([df,df_pca],axis=1)
    df.to_csv("Tech/Daily_PCA/"+stock+".csv",index=0)

for stock in stock_list:
    stock_df = pd.read_csv("Tech/Weekly/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df = df.loc[:,features]
    df = df.dropna()
    df = df.reset_index(drop=True)
    df_pca = knnpca(df)
    df = pd.concat([df,df_pca],axis=1)
    df.to_csv("Tech/Weekly_PCA/"+stock+".csv",index=0)

for stock in stock_list:
    stock_df = pd.read_csv("Tech/Monthly/"+stock+".csv")
    df = stock_df.copy().sort_values(by="Date").reset_index(drop=True)
    df = df.loc[:,features]
    df = df.dropna()
    df = df.reset_index(drop=True)
    df_pca = knnpca(df)
    df = pd.concat([df,df_pca],axis=1)
    df.to_csv("Tech/Monthly_PCA/"+stock+".csv",index=0)

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


Imputing row 1/2537 with 0 missing, elapsed time: 0.502
Imputing row 101/2537 with 0 missing, elapsed time: 0.502
Imputing row 201/2537 with 0 missing, elapsed time: 0.502
Imputing row 301/2537 with 0 missing, elapsed time: 0.502
Imputing row 401/2537 with 0 missing, elapsed time: 0.502
Imputing row 501/2537 with 0 missing, elapsed time: 0.503
Imputing row 601/2537 with 0 missing, elapsed time: 0.503
Imputing row 701/2537 with 0 missing, elapsed time: 0.503
Imputing row 801/2537 with 0 missing, elapsed time: 0.503
Imputing row 901/2537 with 0 missing, elapsed time: 0.503
Imputing row 1001/2537 with 0 missing, elapsed time: 0.503
Imputing row 1101/2537 with 0 missing, elapsed time: 0.503
Imputing row 1201/2537 with 0 missing, elapsed time: 0.503
Imputing row 1301/2537 with 0 missing, elapsed time: 0.503
Imputing row 1401/2537 with 0 missing, elapsed time: 0.504
Imputing row 1501/2537 with 0 missing, elapsed time: 0.504
Imputing row 1601/2537 with 0 missing, elapsed time: 0.504
Imputing 

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


Imputing row 1/2537 with 0 missing, elapsed time: 0.482
Imputing row 101/2537 with 0 missing, elapsed time: 0.482
Imputing row 201/2537 with 0 missing, elapsed time: 0.482
Imputing row 301/2537 with 0 missing, elapsed time: 0.482
Imputing row 401/2537 with 0 missing, elapsed time: 0.482
Imputing row 501/2537 with 0 missing, elapsed time: 0.482
Imputing row 601/2537 with 0 missing, elapsed time: 0.482
Imputing row 701/2537 with 0 missing, elapsed time: 0.482
Imputing row 801/2537 with 0 missing, elapsed time: 0.483
Imputing row 901/2537 with 0 missing, elapsed time: 0.483
Imputing row 1001/2537 with 0 missing, elapsed time: 0.483
Imputing row 1101/2537 with 0 missing, elapsed time: 0.483
Imputing row 1201/2537 with 0 missing, elapsed time: 0.483
Imputing row 1301/2537 with 0 missing, elapsed time: 0.483
Imputing row 1401/2537 with 0 missing, elapsed time: 0.483
Imputing row 1501/2537 with 0 missing, elapsed time: 0.483
Imputing row 1601/2537 with 0 missing, elapsed time: 0.483
Imputing 

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


...

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


Imputing row 1/485 with 0 missing, elapsed time: 0.025
Imputing row 101/485 with 0 missing, elapsed time: 0.025
Imputing row 201/485 with 0 missing, elapsed time: 0.025
Imputing row 301/485 with 0 missing, elapsed time: 0.026
Imputing row 401/485 with 0 missing, elapsed time: 0.026
Imputing row 1/387 with 0 missing, elapsed time: 0.017
Imputing row 101/387 with 0 missing, elapsed time: 0.017
Imputing row 201/387 with 0 missing, elapsed time: 0.017
Imputing row 301/387 with 0 missing, elapsed time: 0.017
Imputing row 1/526 with 0 missing, elapsed time: 0.020
Imputing row 101/526 with 0 missing, elapsed time: 0.020
Imputing row 201/526 with 0 missing, elapsed time: 0.020
Imputing row 301/526 with 0 missing, elapsed time: 0.020
Imputing row 401/526 with 0 missing, elapsed time: 0.020
Imputing row 501/526 with 0 missing, elapsed time: 0.020
Imputing row 1/526 with 0 missing, elapsed time: 0.027
Imputing row 101/526 with 0 missing, elapsed time: 0.028
Imputing row 201/526 with 0 missing, el

/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")


Imputing row 1/121 with 0 missing, elapsed time: 0.003
Imputing row 101/121 with 0 missing, elapsed time: 0.004
Imputing row 1/121 with 0 missing, elapsed time: 0.003
Imputing row 101/121 with 0 missing, elapsed time: 0.004
Imputing row 1/55 with 0 missing, elapsed time: 0.001
Imputing row 1/121 with 0 missing, elapsed time: 0.002
Imputing row 101/121 with 0 missing, elapsed time: 0.002
Imputing row 1/121 with 0 missing, elapsed time: 0.002
Imputing row 101/121 with 0 missing, elapsed time: 0.002
Imputing row 1/121 with 0 missing, elapsed time: 0.001
Imputing row 101/121 with 0 missing, elapsed time: 0.002
Imputing row 1/100 with 0 missing, elapsed time: 0.001
Imputing row 1/77 with 0 missing, elapsed time: 0.001
Imputing row 1/121 with 0 missing, elapsed time: 0.002
Imputing row 101/121 with 0 missing, elapsed time: 0.002
Imputing row 1/121 with 0 missing, elapsed time: 0.001
Imputing row 101/121 with 0 missing, elapsed time: 0.002


/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/solver.py:55: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
/home/heinrich/.local/lib/python3.10/site-packages/fancyimpute/so